# Ionospheric Bubble Probability (IBP)

The ionospheric bubble probability statistical model is a Swarm L2 product, named IBP_CLI. The output of the Ionospheric Bubble Probability (IBP) product is an index, that depends  on the day of year or the month of the year, geographic longitude, local time and solar flux index.
Detailed documentation can be found [here](https://ibp-model.readthedocs.io)

To use this notebook, you will need to install the `ibpmodel` package. The `ibpmodel` package provides functionality for the visualization and analysis of butterfly data.
To install the package, you can run the following command in a code cell:

In [1]:
pip install ibpmodel

Note: you may need to restart the kernel to use updated packages.


## Parameters available for changing

| Parameter | Description | 
|---|---|
|`Month Dropdown`| This dropdown allows you to select a specific month. Changing the month will update the Day of Year (DOY) Slider to reflect the first day of the selected month. 
|`Day of Year (DOY) Slider`| This slider allows you to select a specific day of the year. The value represents the day count from January 1st to December 31st. Changing the DOY will update the Month Dropdown to reflect the corresponding month. Additionally, it will update the plot to display the IBP index for the selected DOY and the current value of the f107 slider.
|`f107 Slider`| This slider represents a solar activity parameter. Changing its value will update the plot to display the IBP index for the selected DOY and the new value of f107. The f107 parameter affects the intensity of the IBP index plot.

## IBP index
This plot visualizes the IBP index based on the selected day of the year (DOY) and solar activity parameter (f107). You can interact with the plot using the following options: `Month Dropdown`, `Day of Year (DOY) Slider`, `f107 Slider`.  
By modifying these options, you can explore different combinations of month, day, and solar activity to observe the variations in the IBP index. The plot is updated in real-time as you change these values.

In [2]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ibpmodel as ibp
from IPython.display import clear_output, display
import numpy as np
import datetime
import bisect

def _cumsum(values):
    sum_value = 0
    for value in values:
        sum_value += value
        yield sum_value

MONTHS_LASTDAYS = list(_cumsum([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]))
MONTHS_LASTDAYS_ARRAY = np.cumsum([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

MONTHS = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December",
]


def doy_to_month_bisect(doy):
    # O(log N) complexity - finds the month in 3-4 iterations
    start_index, end_index = 0, 11
    while start_index != end_index:
        new_index = start_index + (end_index - start_index) // 2
        if doy <= MONTHS_LASTDAYS[new_index]:
            end_index = new_index
        else:
            start_index = new_index + 1
    return start_index

def plot_IBP_index(doy, f107):
    ibp.plotIBPindex(doy=doy, f107=f107, cmap='viridis', alpha=0.9)
    plt.show()

output = widgets.Output()

month_dropdown = widgets.Dropdown(
    options=MONTHS,
    description='Month:'
)

doy_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=365,
    step=1,
    description='Day of year:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%') 
)

solar_slider = widgets.IntSlider(
    value=150,
    min=50,
    max=180,
    step=10,
    description='f107:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%')
)
initial_plot_shown = False

def update_plot(change):
    global initial_plot_shown 
    doy = doy_slider.value
    f107 = solar_slider.value
    with output:
        clear_output(wait=True)
        if not initial_plot_shown:
            plot_IBP_index(doy_slider.value, solar_slider.value)
            initial_plot_shown = False

def update_dropdown(change):
    doy = doy_slider.value
    month = MONTHS[doy_to_month_bisect(doy)]
    month_dropdown.value = month
    if doy_slider.value != doy:
        doy_slider.value = doy

def update_slider(change):
    if month_dropdown.value != MONTHS[doy_to_month_bisect(doy_slider.value)]:
        month = month_dropdown.value
        month_idx = MONTHS.index(month)
        doy_range = range(MONTHS_LASTDAYS[month_idx - 1] + 1, MONTHS_LASTDAYS[month_idx] + 1)
        doy_slider.value = doy_range.start

display(month_dropdown, doy_slider, solar_slider, output)

doy_slider.observe(update_dropdown, names='value')
month_dropdown.observe(update_slider, names='value')
doy_slider.observe(update_plot, names='value')
solar_slider.observe(update_plot, names='value')

update_plot(None)

Dropdown(description='Month:', options=('January', 'February', 'March', 'April', 'May', 'June', 'July', 'Augus…

IntSlider(value=1, continuous_update=False, description='Day of year:', layout=Layout(width='30%'), max=365, m…

IntSlider(value=150, continuous_update=False, description='f107:', layout=Layout(width='30%'), max=180, min=50…

Output()

## IBP index comparation

Here are a few additions and modifications compared to the previous plot. 

The overall functionality of the code remains the same. You can select a `month` from the dropdown, adjust the `day of the year (DOY)` using the slider, and modify the solar activity parameter `f107` with the corresponding slider. The plot is updated only by clicking the `Add Plot` button - the it add plot with newly setted values below default plot. By clicking the `Clear Plots` button it returns you to default plot what gives opportunity to have a new set of plots to compare.

The main difference in this version is the ability to add and compare multiple plots on separate subplots within the same figure.

In [3]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ibpmodel as ibp
from IPython.display import clear_output, display
import numpy as np
import datetime
import bisect

def _cumsum(values):
    sum_value = 0
    for value in values:
        sum_value += value
        yield sum_value

MONTHS_LASTDAYS = list(_cumsum([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]))
MONTHS_LASTDAYS_ARRAY = np.cumsum([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])

MONTHS = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December",
]


def doy_to_month_bisect(doy):
    # O(log N) complexity - finds the month in 3-4 iterations
    start_index, end_index = 0, 11
    while start_index != end_index:
        new_index = start_index + (end_index - start_index) // 2
        if doy <= MONTHS_LASTDAYS[new_index]:
            end_index = new_index
        else:
            start_index = new_index + 1
    return start_index

fig, ax = plt.subplots(constrained_layout=False)
plots = [ax]

initial_plot_shown = False

def plot_IBP_index(doy, f107):
    ax = plt.gca()
    ibp.plotIBPindex(doy=doy, f107=f107, cmap='viridis', alpha=0.9, ax=ax)
    plt.show()

output = widgets.Output()

def update_plot(change):
    global initial_plot_shown
    doy = doy_slider.value
    f107 = solar_slider.value
    with output:
        if not initial_plot_shown:
            plot_IBP_index(doy, f107)
            initial_plot_shown = False

def add_button_clicked(button):
    doy = doy_slider.value
    f107 = solar_slider.value
    with output:
        new_ax = fig.add_subplot(111, sharex=ax, sharey=ax)
        plots.append(new_ax)
        plot_IBP_index(doy, f107)

def clear_button_clicked(button):
    doy_slider.value = 1
    solar_slider.value = 150
    month_dropdown.value = MONTHS[0]
    for plot in plots:
        plot.clear()
    plots.clear()
    plots.append(ax)
    with output:
        clear_output(wait=False)
        plot_IBP_index(1,150)


month_dropdown = widgets.Dropdown(
    options=MONTHS,
    description='Month:'
)

doy_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=365,
    step=1,
    description='Day of year:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%') 
)

solar_slider = widgets.IntSlider(
    value=150,
    min=50,
    max=180,
    step=10,
    description='f107:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%')
)

add_button = widgets.Button(description='Add Plot', button_style='success')
add_button.on_click(add_button_clicked)

clear_button = widgets.Button(description='Clear Plots', button_style='warning')
clear_button.on_click(clear_button_clicked)


def update_dropdown(change):
    doy = doy_slider.value
    month = MONTHS[doy_to_month_bisect(doy)]
    month_dropdown.value = month
    if doy_slider.value != doy:
        doy_slider.value = doy  

    
def update_slider(change):
    if month_dropdown.value != MONTHS[doy_to_month_bisect(doy_slider.value)]:
        month = month_dropdown.value
        month_idx = MONTHS.index(month)
        doy_range = range(MONTHS_LASTDAYS[month_idx - 1] + 1, MONTHS_LASTDAYS[month_idx] + 1)
        doy_slider.value = doy_range.start
        
display(month_dropdown, doy_slider, solar_slider, output,add_button, clear_button)

doy_slider.observe(update_dropdown, names='value')
month_dropdown.observe(update_slider, names='value')

update_plot(None)

Dropdown(description='Month:', options=('January', 'February', 'March', 'April', 'May', 'June', 'July', 'Augus…

IntSlider(value=1, continuous_update=False, description='Day of year:', layout=Layout(width='30%'), max=365, m…

IntSlider(value=150, continuous_update=False, description='f107:', layout=Layout(width='30%'), max=180, min=50…

Output()

Button(button_style='success', description='Add Plot', style=ButtonStyle())

Button(button_style='warning', description='Clear Plots', style=ButtonStyle())

## Plot Butterfly Data

This plot is used to visualize the monthly IBP index over different months. It shows the relationship between the months of the year and the corresponding IBP index values. By changing `f107` slider you can see dependency of IBP index to solar activity.

In [4]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ibpmodel as ibp
from IPython.display import clear_output, display

output = widgets.Output()

def plot_butterfly_data(f107):
    with output:
        clear_output(wait=True)
        ibp.plotButterflyData(f107)

def update_plot(change):
    f107 = solar_slider.value
    with output:
        clear_output(wait=True)
        plot_butterfly_data(f107)

solar_slider = widgets.IntSlider(
    value=150,
    min=50,
    max=180,
    step=10,
    description='f107:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%')
)

display(solar_slider, output)
solar_slider.observe(update_plot, names='value')
plot_butterfly_data(solar_slider.value)


IntSlider(value=150, continuous_update=False, description='f107:', layout=Layout(width='30%'), max=180, min=50…

Output()

## Butterfly Data comparation

Here are a few additions and modifications compared to the previous plot. 

The overall functionality of the code remains the same. You can modify the solar activity parameter `f107` with the corresponding slider. But also you are able to compare data takind different `f107` values. By enabling the `Show multiple plots` checkbox it returns you to set of checkboxes with value of f107 from 50 to 180, enabling/disabling which gives you opportunity to have a new set of plots to compare.

The main difference in this version is the ability to not only change plot in real time but also compare multiple plots on separate subplots within the same figure.

In [5]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ibpmodel as ibp
from IPython.display import clear_output, display
import numpy as np

output = widgets.Output()

def plot_butterfly_data(f107_values, ax=None):
    with output:
        for f107 in f107_values:
            ibp.plotButterflyData(f107, ax=ax)

show_multiple_plots_checkbox = widgets.Checkbox(description='Show multiple plots')
initial_plot_shown = False

def update_plot(f107):
    global initial_plot_shown
    with output:
        clear_output(wait=True)
        if not show_multiple_plots_checkbox.value:
            f107 = solar_slider.value
            solar_slider.layout.visibility = 'visible'
            if not initial_plot_shown:
                plot_butterfly_data([f107])
        else:
            solar_slider.layout.visibility = 'hidden'
            clear_output(wait=True)
            checkboxes = []
            for value in range(50, 181, 10):
                checkbox = widgets.Checkbox(value=False, description=str(value))
                checkbox.layout.width = 'auto' 
                checkboxes.append(checkbox)
            def show_selected_plots(change):
                with output:
                    clear_output(wait=True) 
                    checkboxes_output = widgets.HBox(checkboxes)
                    display(checkboxes_output)
                    selected_values = [int(checkbox.description) for checkbox in checkboxes if checkbox.value]

                    if len(selected_values) > 0:
                        num_plots = len(selected_values)
                        fig, ax = plt.subplots(1, num_plots, figsize=(6*num_plots, 6))
                        if num_plots == 1:
                            ax = [ax]
                        for axs, value in zip(ax, selected_values):
                            plot_butterfly_data([value], axs)  
                    
                    plt.show()

            for checkbox in checkboxes:
                checkbox.observe(show_selected_plots, 'value')
            show_selected_plots(None)
                
                
solar_slider = widgets.IntSlider(
    value=150,
    min=50,
    max=180,
    step=10,
    description='f107:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='30%', visibility='visible')
)

display(show_multiple_plots_checkbox, solar_slider, output)
solar_slider.observe(update_plot, names='value')
show_multiple_plots_checkbox.observe(update_plot, 'value')
plot_butterfly_data([solar_slider.value])

Checkbox(value=False, description='Show multiple plots')

IntSlider(value=150, continuous_update=False, description='f107:', layout=Layout(visibility='visible', width='…

Output()